In [1]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
import networkit as nk
import numpy as np
from tqdm.auto import tqdm
from pandarallel import pandarallel
import pygeos
# import rpyc

pandarallel.initialize(progress_bar=True)
tqdm.pandas()

/home/gk/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
G = nx.read_graphml('/home/gk/Desktop/sasha_morozov/Stockholms kommun_graph.graphml')

In [3]:
'''
Взято из библиотеки
'''


def get_nx2_nk_idmap(G_nx):
    idmap = dict((id, u) for (id, u) in zip(G_nx.nodes(), range(G_nx.number_of_nodes())))
    return idmap

def get_nk_attrs(G_nx):
    attrs = dict(
        (u, {"x": d[-1]["x"], "y": d[-1]["y"]}) 
        for (d, u) in zip(G_nx.nodes(data=True), range(G_nx.number_of_nodes()))
        )
    return attrs

def convert_nx2nk(G_nx, idmap=None, weight=None):

    if not idmap:
        idmap = get_nx2_nk_idmap(G_nx)
    n = max(idmap.values()) + 1
    edges = list(G_nx.edges())

    if weight:
        G_nk = nk.Graph(n, directed=G_nx.is_directed(), weighted=True)
        for u_, v_ in tqdm(edges):
                u, v = idmap[u_], idmap[v_]
                d = dict(G_nx[u_][v_])
                if len(d) > 1:
                    for d_ in d.values():
                            v__ = G_nk.addNodes(2)
                            u__ = v__ - 1
                            w = round(d_[weight], 1) if weight in d_ else 1
                            G_nk.addEdge(u, v, w)
                            G_nk.addEdge(u_, u__, 0)
                            G_nk.addEdge(v_, v__, 0)
                else:
                    d_ = list(d.values())[0]
                    w = round(d_[weight], 1) if weight in d_ else 1
                    G_nk.addEdge(u, v, w)
    else:
        G_nk = nk.Graph(n, directed=G_nx.is_directed())
        for u_, v_ in edges:
                u, v = idmap[u_], idmap[v_]
                G_nk.addEdge(u, v)

    return G_nk

In [4]:
ebunch = list(((u, v) for u,v,e in G.edges(data=True) if e['type'] == 'car'))
G.remove_edges_from(ebunch)

In [5]:
G_nx = nx.convert_node_labels_to_integers(G)
G_nk = convert_nx2nk(G_nx, weight='time_min')
# G_nx = G

100%|██████████| 164010/164010 [00:01<00:00, 92199.26it/s]


In [6]:
graph_df = pd.DataFrame.from_dict(dict(G_nx.nodes(data=True)), orient='index')
graph_df

,x,y,stop,desc
0,330542.57,6580425.30,False,NaN
1,330437.44,6580405.70,False,NaN
2,330676.27,6580278.54,False,NaN
3,330606.49,6580394.67,False,NaN
4,330879.06,6580779.06,False,NaN
...,...,...,...,...
58946,333807.26,6576867.99,True,bus
58947,334861.70,6575748.52,True,bus
58948,334783.19,6575266.20,True,bus
58949,335144.94,6574936.28,True,bus


In [7]:
graph_gdf = gpd.GeoDataFrame(graph_df, geometry =gpd.points_from_xy(graph_df['x'], graph_df['y']), crs = 32634)

In [8]:
# graph_gdf = graph_gdf.to_crs(4326)

In [18]:
# graph_gdf.to_file('AMSTG.geojson')

In [9]:
blocks = gpd.read_file('districts_splitted_Stockholms kommun.geojson')

In [10]:
blocks.crs

<Projected CRS: EPSG:32634>
Name: WGS 84 / UTM zone 34N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 18°E and 24°E, northern hemisphere between equator and 84°N, onshore and offshore. Albania. Belarus. Bosnia and Herzegovina. Bulgaria. Central African Republic. Chad. Croatia. Democratic Republic of the Congo (Zaire). Estonia. Finland. Greece. Hungary. Italy. Kosovo. Latvia. Libya. Lithuania. Montenegro. North Macedonia. Norway, including Svalbard and Bjornoys. Poland. Romania. Russian Federation. Serbia. Slovakia. Sudan. Sweden. Ukraine.
- bounds: (18.0, 0.0, 24.0, 84.0)
Coordinate Operation:
- name: UTM zone 34N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
# blocks = blocks.to_crs(4326)

In [11]:
# blocks.drop(columns=['area'], inplace=True)
blocks.reset_index(inplace=True)
blocks.rename(columns={'index':'id'}, inplace=True)
blocks['centroids'] = blocks['geometry'].centroid
blocks.drop(columns=['geometry'], inplace=True)
blocks.rename(columns={'centroids':'geometry'}, inplace=True)

In [12]:
blocks.head()

,id,geometry
0,0,POINT (336814.810 6574219.259)
1,1,POINT (337389.332 6574168.700)
2,2,POINT (337028.449 6574146.297)
3,3,POINT (337368.863 6574104.289)
4,4,POINT (337297.898 6573930.810)


In [13]:
from_blocks = graph_gdf['geometry'].sindex.nearest(blocks['geometry'], return_distance = False, return_all = False)

In [14]:
from_blocks

array([[    0,     1,     2, ...,  6009,  6010,  6011],
       [22376, 58220, 17067, ..., 58705, 58705, 58705]])

In [15]:
blocks.shape

(6012, 2)

In [16]:
Matrix = pd.DataFrame(0, index = from_blocks[1], 
                        columns = from_blocks[1])
nodes = from_blocks[1]

In [15]:
# t = 337126
# s = pd.DataFrame()
# l = list()
# nk_dists = nk.distance.SPSP(G_nk, sources = [t]).run()
# for b in tqdm(Matrix.index):
#     x = nk_dists.getDistance(t, b)
#     l.append(x)
#     print(x, end='\r')

In [17]:
blocks.explore()

In [18]:
def get_nk_distances(nk_dists, loc):

    target_nodes = loc.index
    source_node = loc.name
    distances = [nk_dists.getDistance(source_node, node) for node in target_nodes]

    return pd.Series(data = distances, index = target_nodes)


nk_dists = nk.distance.SPSP(G_nk, sources = Matrix.index.values).run()

Matrix =  Matrix.apply(lambda x: get_nk_distances(nk_dists, x), axis =1)
Matrix.index = blocks['id']
Matrix.columns = blocks['id']

In [19]:
Matrix

id,0,1,2,3,4,5,6,7,8,9,...,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,10.2,15.0,10.8,14.8,11.6,8.4,14.3,11.2,9.0,...,183.3,119.8,186.3,187.0,187.0,188.7,121.9,130.6,130.6,130.6
1,9.3,0.0,5.6,0.6,5.0,6.4,3.2,4.1,7.1,5.2,...,183.9,120.4,186.9,187.6,187.6,189.3,122.5,131.2,131.2,131.2
2,14.0,5.6,0.0,6.0,8.2,8.0,6.6,1.5,8.7,8.6,...,188.8,125.3,191.8,192.5,192.5,194.2,127.4,136.1,136.1,136.1
3,9.9,0.6,6.0,0.0,4.7,5.8,2.6,4.5,6.5,4.6,...,184.5,121.0,187.5,188.2,188.2,189.9,123.1,131.8,131.8,131.8
4,13.8,5.0,8.2,4.7,0.0,4.6,6.4,9.1,8.5,8.4,...,188.9,125.4,191.9,192.6,192.6,194.3,127.5,136.2,136.2,136.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6007,187.3,188.4,192.3,187.8,192.5,193.6,190.4,190.8,193.3,191.1,...,192.0,141.6,195.0,195.7,195.7,0.0,143.7,158.2,158.2,158.2
6008,184.0,185.1,189.0,184.5,189.2,190.3,187.1,187.5,190.0,187.8,...,188.7,138.3,191.7,192.4,192.4,194.1,0.0,154.9,154.9,154.9
6009,112.2,113.3,117.2,112.7,117.4,118.5,115.3,115.7,118.2,116.0,...,135.6,73.3,138.6,139.3,139.3,141.0,75.4,0.0,0.0,0.0


In [20]:
Matrix.to_pickle('matrix_mmg_Stockholms kommun.pkl')